# Molecular Vibration Visualization - Basics

This notebook demonstrates how to visualize molecular vibrations from quantum chemistry calculations using plotlyMol.

## What You'll Learn

1. Parsing vibrational data from Gaussian, ORCA, or Molden files
2. Visualizing vibrations with static displacement arrows
3. Creating animated vibrations
4. Coloring atoms by displacement magnitude (heatmap)
5. Accessing vibrational mode data

## Prerequisites

```bash
pip install plotlymol
```

## Setup

In [ ]:
# Import required libraries
from plotlymol3d import (
    parse_vibrations,
    draw_3D_rep,
    add_vibrations_to_figure,
    create_vibration_animation,
    create_displacement_arrows,
    create_heatmap_colored_figure,
)
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

## 1. Parsing Vibrational Data

plotlyMol supports three quantum chemistry file formats:
- **Gaussian** (.log files)
- **ORCA** (.out files)
- **Molden** (.molden files)

The `parse_vibrations()` function automatically detects the format.

In [ ]:
# Example: Parse a Gaussian frequency calculation
# Replace with your own file path
vib_file = "path/to/your/calculation.log"  # or .out or .molden

# Parse the vibration data
vib_data = parse_vibrations(vib_file)

print(f"Program: {vib_data.program}")
print(f"Number of atoms: {len(vib_data.atomic_numbers)}")
print(f"Number of vibrational modes: {len(vib_data.modes)}")

## 2. Exploring Vibrational Modes

Each mode contains frequency, IR intensity (if available), and displacement vectors.

In [ ]:
# List all modes with their frequencies
print("\nVibrational Modes:")
print("-" * 60)
for mode in vib_data.modes:
    freq_str = f"{mode.frequency:.1f} cm⁻¹"
    if mode.is_imaginary:
        freq_str += " (imaginary)"
    
    if mode.ir_intensity is not None:
        print(f"Mode {mode.mode_number}: {freq_str:>20} | IR: {mode.ir_intensity:.1f} km/mol")
    else:
        print(f"Mode {mode.mode_number}: {freq_str:>20}")

## 3. Static Displacement Arrows

Visualize displacement vectors as 3D arrows showing how atoms move during vibration.

In [ ]:
# Quick method: Use draw_3D_rep with vibration parameters
fig = draw_3D_rep(
    smiles="O",  # Replace with your molecule's SMILES
    vibration_file=vib_file,
    vibration_mode=1,  # First vibrational mode
    vibration_display="arrows",
    vibration_amplitude=1.5,  # Amplitude multiplier
    mode="ball+stick"
)

fig.show()

### Customizing Arrow Visualization

In [ ]:
# Advanced method: Create figure first, then add vibrations
fig = draw_3D_rep(smiles="O", mode="ball+stick")

fig = add_vibrations_to_figure(
    fig=fig,
    vib_data=vib_data,
    mode_number=1,
    display_type="arrows",
    amplitude=2.0,  # Larger amplitude
    arrow_scale=1.5,  # Larger arrows
    arrow_color="#FF6B35",  # Custom color (orange)
)

fig.show()

## 4. Animated Vibrations

Create an interactive animation showing the vibrational motion with play/pause controls.

In [ ]:
# Create RDKit molecule from SMILES
mol = Chem.MolFromSmiles("O")  # Replace with your SMILES
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, randomSeed=42)

# Create animation
fig = create_vibration_animation(
    vib_data=vib_data,
    mode_number=1,
    mol=mol,
    amplitude=0.5,  # Smaller amplitude for smoother animation
    n_frames=30,  # More frames = smoother (but slower)
    mode="ball+stick",
    resolution=32  # Sphere quality
)

fig.show()

## 5. Heatmap Coloring by Displacement

Color atoms by their displacement magnitude to quickly identify which atoms move most.

In [ ]:
# Create molecular figure
fig = draw_3D_rep(smiles="O", mode="ball+stick")

# Add heatmap coloring
fig = add_vibrations_to_figure(
    fig=fig,
    vib_data=vib_data,
    mode_number=1,
    display_type="heatmap",
    heatmap_colorscale="Reds",  # Try: Blues, Viridis, Plasma, Hot
    show_colorbar=True
)

fig.show()

### Combining Arrows and Heatmap

In [ ]:
# Show both displacement arrows AND heatmap coloring
fig = draw_3D_rep(smiles="O", mode="ball+stick")

fig = add_vibrations_to_figure(
    fig=fig,
    vib_data=vib_data,
    mode_number=1,
    display_type="both",  # Arrows + heatmap
    amplitude=1.5,
    arrow_color="#000000",  # Black arrows
    heatmap_colorscale="YlOrRd",
    show_colorbar=True
)

fig.show()

## 6. Accessing Displacement Data

Extract displacement magnitudes for quantitative analysis.

In [ ]:
# Get displacement magnitudes for a specific mode
mode_number = 1
displacements = vib_data.get_displacement_magnitudes(mode_number)

print(f"\nDisplacement magnitudes for Mode {mode_number}:")
print("-" * 60)
for i, disp in enumerate(displacements):
    atom_num = vib_data.atomic_numbers[i]
    print(f"Atom {i+1} (Z={atom_num}): {disp:.4f} Å")

print(f"\nMax displacement: {np.max(displacements):.4f} Å")
print(f"Average displacement: {np.mean(displacements):.4f} Å")

## 7. Comparing Multiple Modes

In [ ]:
# Visualize the three most intense IR modes
# Sort modes by IR intensity (if available)
modes_with_ir = [m for m in vib_data.modes if m.ir_intensity is not None]

if modes_with_ir:
    sorted_modes = sorted(modes_with_ir, key=lambda m: m.ir_intensity, reverse=True)
    top_3 = sorted_modes[:3]
    
    print("\nTop 3 IR-active modes:")
    for mode in top_3:
        print(f"Mode {mode.mode_number}: {mode.frequency:.1f} cm⁻¹ (IR: {mode.ir_intensity:.1f})")
        
        # Visualize each mode
        fig = draw_3D_rep(smiles="O", mode="ball+stick")
        fig = add_vibrations_to_figure(
            fig=fig,
            vib_data=vib_data,
            mode_number=mode.mode_number,
            display_type="both",
            amplitude=1.5,
            heatmap_colorscale="Reds"
        )
        fig.show()
else:
    print("No IR intensities available in this file")

## Summary

In this notebook, you learned:

✅ How to parse vibrational data from Gaussian, ORCA, or Molden files
✅ Three visualization modes: arrows, animation, heatmap
✅ How to customize visualization parameters
✅ How to access displacement data programmatically
✅ How to compare multiple vibrational modes

## Next Steps

- Try the **Advanced Vibration Analysis** notebook for more complex workflows
- Explore different colorscales and amplitude settings
- Export figures as static images or HTML for sharing
- Analyze mode contributions to IR spectra